In [1]:
# import packages

import dask_awkward as dak
import numpy as np
from coffea.dataset_tools import preprocess
from distributed import Client
from matplotlib import pyplot as plt

import egamma_tnp
from egamma_tnp import ElectronTagNProbeFromNanoAOD, ElectronTagNProbeFromNTuples
from egamma_tnp.plot import plot_efficiency

In [3]:
# start client
client = Client()

/usr/local/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45551 instead
  warnings.warn(


In [5]:
# define datasets

fileset = {
    "samples": {
        "files": {
            "../root_files/Data_Run2023C_EGamma0_NANOAOD_22Sep2023_v1-v1_001.root": "Events",
        }
    }
}

print(fileset)

fileset_available, fileset_updates = preprocess(fileset, skip_bad_files=True, step_size=500_000)

{'samples': {'files': {'../root_files/Data_Run2023C_EGamma0_NANOAOD_22Sep2023_v1-v1_001.root': 'Events'}}}


In [17]:
# initialize tag and probe class

tnp = ElectronTagNProbeFromNanoAOD(
    fileset_available,
    filters=["HLT_Ele30_WPTight_Gsf"],
    filterbit=[1],
    tags_pt_cut=30,
    use_sc_eta=True,
    tags_abseta_cut=2.50,
    probes_pt_cut=27,
    trigger_pt=[30],
)

egamma_tnp.binning.set("MET_pt_bins", np.linspace(0, 200, 21).tolist())
egamma_tnp.binning.set("luminosityBlock_bins", np.linspace(0, 1000, 11).tolist())
egamma_tnp.binning.set("tag_Ele_pt_bins", egamma_tnp.binning.get("pt_bins"))
egamma_tnp.binning.set("Electron_pt_bins", egamma_tnp.binning.get("pt_bins"))

In [15]:
# get arrays
out = tnp.get_tnp_arrays(cut_and_count=True, vars=["Electron_pt", "tag_Ele_eta", "el_pt", "el_eta", "MET_pt", "luminosityBlock"], compute=True)

/usr/local/lib/python3.11/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [16]:
out["samples"]["passing"]

FieldNotFoundError: no field 'passing' in record with 7 fields